In [1]:
import numpy as np
import json, geojson, overpy, pyproj, logging
from shapely import geometry
from shapely.ops import transform,linemerge, unary_union, polygonize, cascaded_union
from functools import partial
import matplotlib.pyplot as plt

logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )



In [16]:
api = overpy.Overpass()

In [18]:
api.url

'http://overpass-api.de/api/interpreter'

In [17]:
dir(api)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_regex_extract_error_msg',
 '_regex_remove_tag',
 'default_read_chunk_size',
 'default_url',
 'parse_json',
 'parse_xml',
 'query',
 'read_chunk_size',
 'url',
 'xml_parser']

In [2]:

class LabelCube:
    ### Get the data
    ### Get the map
    ### Get a google basemap or something

    def __init__(self, json_path, resolution, side_length):
        self.resolution = resolution   #m/pix
        self.side_length = side_length  #m
        self.labels_json = json.load(open(json_path,'r'))
        WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
        self.wgs_proj = pyproj.Proj(WGS84)
        self.api = overpy.Overpass()
        self.json_spec = json.load(open(json_path,'r'))

    def get_utm_zone(self, lat,lon):
        zone_str = str(int((lon + 180)/6) + 1)

        if ((lat>=56.) & (lat<64.) & (lon >=3.) & (lon <12.)):
            zone_str = '32'
        elif ((lat >= 72.) & (lat <84.)):
            if ((lon >=0.) & (lon<9.)):
                zone_str = '31'
            elif ((lon >=9.) & (lon<21.)):
                zone_str = '33'
            elif ((lon >=21.) & (lon<33.)):
                zone_str = '35'
            elif ((lon >=33.) & (lon<42.)):
                zone_str = '37'

        return zone_str

    def _set_utm_proj(self,lat,lon):
        self.utm_zone = self.get_utm_zone(lat,lon)
        self.utm_proj = pyproj.Proj(proj='utm', zone=self.utm_zone, ellps='WGS84')
        self.projection_for = partial(pyproj.transform, self.wgs_proj, self.utm_proj)
        self.projection_rev = partial(pyproj.transform, self.utm_proj, self.wgs_proj)


    def _get_bbox(self,lat,lon):

        pt_utm = transform(self.projection_for,geometry.Point(lat,lon))

        bbox_utm = geometry.box(pt_utm.x-self.side_length/2,
                                pt_utm.y-self.side_length/2,
                                pt_utm.x+self.side_length/2,
                                pt_utm.y+self.side_length/2)

        self.bbox_ll = transform(self.projection_rev, bbox_utm)
        #bb = self.bbox_ll.bounds

        #(bb[1],bb[0],bb[3],bb[2])
        return self.bbox_ll.bounds


    def _parse_osm(self, response):
        lss = []
        for ii_w,way in enumerate(response.ways):
            ls_coords = []
            print('way',ii_w)
            for node in way.nodes:
                ls_coords.append((node.lon,node.lat))
            lss.append(geometry.LineString(ls_coords))

        merged = linemerge([*lss])
        borders = unary_union(merged)
        polygons = list(polygonize(borders))
        big_geom = cascaded_union(polygons)

        if big_geom.type=='MultiPolygon':
            return list(big_geom)
        elif big_geom.type=='Polygon':
            return [big_geom]

    def visualise_results(self):
        fig, ax = plt.subplots(1,1,figsize=(12,12))
        for kk,vv in self.osm_results.items():
            rand_col = list(np.random.choice(range(255), size=3)/255)
            for pp in vv:
                xs, ys = pp.exterior.xy
                ax.plot(xs,ys,c=rand_col)
            for hole in pp.interiors:
                xs,ys = hole.xy
                ax.plot(xs,ys,c=rand_col)

        plt.show()




    def gen_cube(self, lat, lon):
        self._set_utm_proj(lat,lon)
        bbox = self._get_bbox(lat,lon)

        self.osm_results = {}
        self.responses = []

        for kk,vv in self.json_spec.items():
            logging.info(f"class key: {kk}, tags: {vv}")
            self.osm_results[kk] = []
            for item in vv:
                logging.info(f"item: {item}")

                query = """[out:json];
                        (
                        node[{item}]{bbox};
                        way[{item}]{bbox};
                        rel[{item}]{bbox};
                        );
                        out geom;
                        >;
                        out skel qt;
                        """.format(bbox=str(bbox),item=item)

                logging.info(f'query:{query}')

                response = self.api.query(query)
                self.responses.append(response)
                print ('response')
                print (dir(response))
                logging.info(f'n nodes: {len(response.nodes)}, n ways: {len(response.ways)}, n relations: {len(response.relations)}')
                self.osm_results[kk] += self._parse_osm(response)

In [3]:
labelcube = LabelCube('json_zoo/demo.json',10,350)

In [4]:
labelcube.gen_cube(51.750084, -1.244094)

2019-11-30 17:05:59 - INFO - class key: water, tags: ['"natural"="water"']
2019-11-30 17:05:59 - INFO - item: "natural"="water"
2019-11-30 17:05:59 - INFO - query:[out:json];
                        (
                        node["natural"="water"](51.74915420063536, -1.2450298622813625, 51.75101379873122, -1.2431581786531694);
                        way["natural"="water"](51.74915420063536, -1.2450298622813625, 51.75101379873122, -1.2431581786531694);
                        rel["natural"="water"](51.74915420063536, -1.2450298622813625, 51.75101379873122, -1.2431581786531694);
                        );
                        out geom;
                        >;
                        out skel qt;
                        
2019-11-30 17:06:01 - INFO - n nodes: 992, n ways: 14, n relations: 1
2019-11-30 17:06:01 - INFO - class key: building, tags: ['"building"']
2019-11-30 17:06:01 - INFO - item: "building"
2019-11-30 17:06:01 - INFO - query:[out:json];
                        (
    

response
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_areas', '_class_collection_map', '_nodes', '_relations', '_ways', 'api', 'append', 'area_ids', 'areas', 'expand', 'from_json', 'from_xml', 'get_area', 'get_area_ids', 'get_areas', 'get_elements', 'get_ids', 'get_node', 'get_node_ids', 'get_nodes', 'get_relation', 'get_relation_ids', 'get_relations', 'get_way', 'get_way_ids', 'get_ways', 'node_ids', 'nodes', 'relation_ids', 'relations', 'way_ids', 'ways']
way 0
way 1
way 2
way 3
way 4
way 5
way 6
way 7
way 8
way 9
way 10
way 11
way 12
way 13


2019-11-30 17:06:02 - INFO - n nodes: 313, n ways: 28, n relations: 1


response
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_areas', '_class_collection_map', '_nodes', '_relations', '_ways', 'api', 'append', 'area_ids', 'areas', 'expand', 'from_json', 'from_xml', 'get_area', 'get_area_ids', 'get_areas', 'get_elements', 'get_ids', 'get_node', 'get_node_ids', 'get_nodes', 'get_relation', 'get_relation_ids', 'get_relations', 'get_way', 'get_way_ids', 'get_ways', 'node_ids', 'nodes', 'relation_ids', 'relations', 'way_ids', 'ways']
way 0
way 1
way 2
way 3
way 4
way 5
way 6
way 7
way 8
way 9
way 10
way 11
way 12
way 13
way 14
way 15
way 16
way 17
way 18
way 19
way 20
way 21
way 22
way 23
way 24
way 25
way 26
way 27


In [13]:
labelcube.responses[0].__str__

<method-wrapper '__str__' of Result object at 0x7fe5008e1748>

In [14]:
geojson.dumps(labelcube.responses[0])

TypeError: Object of type Result is not JSON serializable

In [63]:
def reduce_member_geoms(members):
    lss = []
    for member in members:
        ls_coords = []
        for geom in member.geometry:
            ls_coords.append((geom.lon,geom.lat))
        lss.append(geometry.LineString(ls_coords))
        
    merged = linemerge([*lss])

    borders = unary_union(merged)

    polygons = list(polygonize(borders))

    big_geom = cascaded_union(polygons)
    
    return big_geom
        

def parse_relation_geom(members):
    outer_mems = [mem for mem in members if mem.role in ['outer','part']]
    inner_mems = [mem for mem in members if mem.role=='inner']
    
    outer_geom = reduce_member_geoms(outer_mems)
    inner_geom = reduce_member_geoms(inner_mems)
    
    rel_geom = outer_geom.difference(inner_geom)
    
    return rel_geom
    
#print (labelcube.responses[0].relations[0].members[0].geometry)
rel_geom = parse_relation_geom(labelcube.responses[1].relations[0].members)

print (rel_geom.type)

Polygon


In [70]:
labelcube.responses[0].ways[0]

<overpy.Way id=46671729 nodes=[597186234, 597186231, 597186230, 3897201548, 5480637471, 920273878, 920273835, 596422858, 597186217, 5480633019, 1330548217, 5480633020, 597186218, 597186220, 597186222, 5480639124, 5480639123, 597186221, 597186210, 597186224, 458160256, 458160280, 2973379294, 5480637469, 1872483274, 3669237192, 29548830, 8089064, 597186238, 915580566, 8089092, 915580668, 8089093, 915580721, 8089094, 915580800, 915580902, 915580631, 915580811, 915580886, 915580650, 8089096, 8089097, 915580587, 915580641, 915580733, 915580843, 29548887, 915580783, 915580914, 915580639, 915580760, 915580614, 915580836, 915580874, 915580595, 915580599, 8089101, 2495602999, 920199507, 920199436, 920199652, 920199617, 920199532, 920199460, 920199413, 920199472, 920199420, 920199605, 5480614956, 920199518, 920199490, 920199440, 920199629, 920199558, 920199582, 920199616, 920199412, 920199410, 920199526, 920199585, 920199516, 5480588153, 5480588152, 920199626, 920199418, 920199533, 920199618, 92

In [66]:
for r in labelcube.responses[0].relations:
    for m in r.members:
        print (m.attributes)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [15]:
print (labelcube.responses[0])

In [32]:
labelcube.responses[0].relations[0].members[0]

<overpy.RelationWay ref=176982921 role=outer>

In [29]:
dir(labelcube.responses[0].relations[0].members[0].geometry[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'lat',
 'lon']

In [22]:
dir(labelcube.responses[0].relations[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attribute_modifiers',
 '_result',
 '_type_value',
 'attributes',
 'center_lat',
 'center_lon',
 'from_json',
 'from_xml',
 'get_center_from_json',
 'get_center_from_xml_dom',
 'id',
 'members',
 'tags']